In [3]:
import os
import csv
import platform
import pandas as pd
import networkx as nx
from graph_partitioning import GraphPartitioning, utils

run_metrics = True

cols = ["WASTE", "CUT RATIO", "EDGES CUT", "TOTAL COMM VOLUME", "Qds", "CONDUCTANCE", "MAXPERM", "RBSE", "NMI", "FSCORE", "FSCORE RELABEL IMPROVEMENT", "LONELINESS"]
#cols = ["WASTE", "CUT RATIO", "EDGES CUT", "TOTAL COMM VOLUME", "Q", "Qds", "CONDUCTANCE", "LONELINESS", "NETWORK PERMANENCE", "NORM. MUTUAL INFO", "EDGE CUT WEIGHT", "FSCORE", "FSCORE RELABEL IMPROVEMENT"]
#cols = ["WASTE", "CUT RATIO", "EDGES CUT", "TOTAL COMM VOLUME", "MODULARITY", "LONELINESS", "NETWORK PERMANENCE", "NORM. MUTUAL INFO", "EDGE CUT WEIGHT", "FSCORE", "FSCORE RELABEL IMPROVEMENT"]

pwd = %pwd

config = {

    "DATA_FILENAME": os.path.join(pwd, "data", "predition_model_tests", "network", "network_$$.txt"),
    "OUTPUT_DIRECTORY": os.path.join(pwd, "output"),

    # Set which algorithm is run for the PREDICTION MODEL.
    # Either: 'FENNEL' or 'SCOTCH'
    "PREDICTION_MODEL_ALGORITHM": "FENNEL",

    # Alternativly, read input file for prediction model.
    # Set to empty to generate prediction model using algorithm value above.
    "PREDICTION_MODEL": "",

    
    "PARTITIONER_ALGORITHM": "FENNEL",

    # File containing simulated arrivals. This is used in simulating nodes
    # arriving at the shelter. Nodes represented by line number; value of
    # 1 represents a node as arrived; value of 0 represents the node as not
    # arrived or needing a shelter.
    "SIMULATED_ARRIVAL_FILE": os.path.join(pwd,
                                           "data",
                                           "predition_model_tests",
                                           "dataset_1_shift_rotate",
                                           "simulated_arrival_list",
                                           "percentage_of_prediction_correct_100",
                                           "arrival_100_$$.txt"
                                          ),

    # File containing the prediction of a node arriving. This is different to the
    # simulated arrivals, the values in this file are known before the disaster.
    "PREDICTION_LIST_FILE": os.path.join(pwd,
                                         "data",
                                         "predition_model_tests",
                                         "dataset_1_shift_rotate",
                                         "prediction_list",
                                         "prediction_$$.txt"
                                        ),

    # File containing the geographic location of each node, in "x,y" format.
    "POPULATION_LOCATION_FILE": os.path.join(pwd,
                                             "data",
                                             "predition_model_tests",
                                             "coordinates",
                                             "coordinates_$$.txt"
                                            ),

    # Number of shelters
    "num_partitions": 4,

    # The number of iterations when making prediction model
    "num_iterations": 1,

    # Percentage of prediction model to use before discarding
    # When set to 0, prediction model is discarded, useful for one-shot
    "prediction_model_cut_off": .0,

    # Alpha value used in one-shot (when restream_batches set to 1)
    "one_shot_alpha": 0.5,
    
    "use_one_shot_alpha" : False,
    
    # Number of arrivals to batch before recalculating alpha and restreaming.
    "restream_batches": 50,

    # When the batch size is reached: if set to True, each node is assigned
    # individually as first in first out. If set to False, the entire batch
    # is processed and empty before working on the next batch.
    "sliding_window": False,

    # Create virtual nodes based on prediction model
    "use_virtual_nodes": False,

    # Virtual nodes: edge weight
    "virtual_edge_weight": 1.0,
    
    # Loneliness score parameter. Used when scoring a partition by how many
    # lonely nodes exist.
    "loneliness_score_param": 1.2,


    ####
    # GRAPH MODIFICATION FUNCTIONS

    # Also enables the edge calculation function.
    "graph_modification_functions": True,

    # If set, the node weight is set to 100 if the node arrives at the shelter,
    # otherwise the node is removed from the graph.
    "alter_arrived_node_weight_to_100": False,

    # Uses generalized additive models from R to generate prediction of nodes not
    # arrived. This sets the node weight on unarrived nodes the the prediction
    # given by a GAM.
    # Needs POPULATION_LOCATION_FILE to be set.
    "alter_node_weight_to_gam_prediction": False,
    
    # Enables edge expansion when graph_modification_functions is set to true
    "edge_expansion_enabled": True,

    # The value of 'k' used in the GAM will be the number of nodes arrived until
    # it reaches this max value.
    "gam_k_value": 100,

    # Alter the edge weight for nodes that haven't arrived. This is a way to
    # de-emphasise the prediction model for the unknown nodes.
    "prediction_model_emphasis": 1.0,
    
    # This applies the prediction_list_file node weights onto the nodes in the graph
    # when the prediction model is being computed and then removes the weights
    # for the cutoff and batch arrival modes
    "apply_prediction_model_weights": True,

    "SCOTCH_LIB_PATH": os.path.join(pwd, "libs/scotch/macOS/libscotch.dylib")
    if 'Darwin' in platform.system()
    else "/usr/local/lib/libscotch.so",
    
    # Path to the PaToH shared library
    "PATOH_LIB_PATH": os.path.join(pwd, "libs/patoh/lib/macOS/libpatoh.dylib")
    if 'Darwin' in platform.system()
    else os.path.join(pwd, "libs/patoh/lib/linux/libpatoh.so"),
    
    "PATOH_ITERATIONS": 5,
        
    # Expansion modes: 'avg_node_weight', 'total_node_weight', 'smallest_node_weight'
    # 'largest_node_weight'
    # add '_squared' or '_sqrt' at the end of any of the above for ^2 or sqrt(weight)
    # i.e. 'avg_node_weight_squared
    "PATOH_HYPEREDGE_EXPANSION_MODE": 'no_expansion',
    
    # Edge Expansion: average, total, minimum, maximum, product, product_squared, sqrt_product
    "EDGE_EXPANSION_MODE" : 'total',
    
    # Whether nodes should be reordered using a centrality metric for optimal node assignments in batch mode
    # This is specific to FENNEL and at the moment Leverage Centrality is used to compute new noder orders
    "FENNEL_NODE_REORDERING_ENABLED": False,
    
    # Whether the Friend of a Friend scoring system is active during FENNEL partitioning.
    # FOAF employs information about a node's friends to determine the best partition when
    # this node arrives at a shelter and no shelter has friends already arrived
    "FENNEL_FRIEND_OF_A_FRIEND_ENABLED": False,
    
    # Alters how much information to print. Keep it at 1 for this notebook.
    # 0 - will print nothing, useful for batch operations.
    # 1 - prints basic information on assignments and operations.
    # 2 - prints more information as it batches arrivals.
    "verbose": 1
}

gp = GraphPartitioning(config)

# Optional: shuffle the order of nodes arriving
# Arrival order should not be shuffled if using GAM to alter node weights
#random.shuffle(gp.arrival_order)

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [4]:
import scipy
from copy import deepcopy
import time

iterations = 100

num_partitions = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

for mode in num_partitions:

    metricsDataPrediction = []
    metricsDataAssign = []
    
    dataQdsOv = []
    dataCondOv = []
        
    config['num_partitions'] = mode
    
    print('Mode', mode)
    elapsed_times = []
    for i in range(0, iterations):
        # how many networks
        if (i % 20) == 0:
            print('Mode', mode, 'Iteration', str(i))
        
        conf = deepcopy(config)

        conf["DATA_FILENAME"] = conf["DATA_FILENAME"].replace('$$', str(i + 1))
        conf["SIMULATED_ARRIVAL_FILE"] = conf["SIMULATED_ARRIVAL_FILE"].replace('$$', str(i + 1))
        conf["PREDICTION_LIST_FILE"] = conf["PREDICTION_LIST_FILE"].replace('$$', str(i + 1))
        conf["POPULATION_LOCATION_FILE"] = conf["POPULATION_LOCATION_FILE"].replace('$$', str(i + 1))


        with GraphPartitioning(conf) as gp:
            gp.verbose = 0
            
            start_time = time.time()
            
            gp.load_network()
            gp.init_partitioner()

            m = gp.prediction_model()
            m = gp.assign_cut_off()
            m = gp.batch_arrival()

            elapsed_time = time.time() - start_time
            elapsed_times.append(elapsed_time)
            
            totalM = len(m)
            metricsDataPrediction.append(m[totalM - 1])
    
    print('Average Elapsed Time =', scipy.mean(elapsed_times))

    waste = ''
    cutratio = ''
    ec = ''
    tcv = ''
    qds = ''
    conductance = ''
    maxperm = ''
    rbse = ''
    nmi = ''
    lonliness = ''
    fscore = ''
    fscoreimprove = ''
        
    qdsOv = ''
    condOv = ''

    dataWaste = []
    dataCutRatio = []
    dataEC = []
    dataTCV = [] 
    dataQDS = []
    dataCOND = []
    dataMAXPERM = []
    dataRBSE = []
    dataNMI = []
    dataLonliness = []
    dataFscore = []
    dataFscoreImprove = []

    
    for i in range(0, iterations):
        dataWaste.append(metricsDataPrediction[i][0])        
        dataCutRatio.append(metricsDataPrediction[i][1])
        dataEC.append(metricsDataPrediction[i][2])
        dataTCV.append(metricsDataPrediction[i][3])
        dataQDS.append(metricsDataPrediction[i][4])
        dataCOND.append(metricsDataPrediction[i][5])
        dataMAXPERM.append(metricsDataPrediction[i][6])
        dataRBSE.append(metricsDataPrediction[i][7])
        dataNMI.append(metricsDataPrediction[i][8])        
        dataFscore.append(metricsDataPrediction[i][9])        
        dataFscoreImprove.append(metricsDataPrediction[i][10])        
        dataLonliness.append(metricsDataPrediction[i][11])


        if(len(waste)):
            waste = waste + ','
        waste = waste + str(metricsDataPrediction[i][0])

        if(len(cutratio)):
            cutratio = cutratio + ','
        cutratio = cutratio + str(metricsDataPrediction[i][1])

        if(len(ec)):
            ec = ec + ','
        ec = ec + str(metricsDataPrediction[i][2])
        
        if(len(tcv)):
            tcv = tcv + ','
        tcv = tcv + str(metricsDataPrediction[i][3])

        if(len(qds)):
            qds = qds + ','
        qds = qds + str(metricsDataPrediction[i][4])

        if(len(conductance)):
            conductance = conductance + ','
        conductance = conductance + str(metricsDataPrediction[i][5])

        if(len(maxperm)):
            maxperm = maxperm + ','
        maxperm = maxperm + str(metricsDataPrediction[i][6])

        if(len(rbse)):
            rbse = rbse + ','
        rbse = rbse + str(metricsDataPrediction[i][7])


        
        if(len(nmi)):
            nmi = nmi + ','
        nmi = nmi + str(metricsDataPrediction[i][8])

        if(len(fscore)):
            fscore = fscore + ','
        fscore = fscore + str(metricsDataPrediction[i][9])

        if(len(fscoreimprove)):
            fscoreimprove = fscoreimprove + ','
        fscoreimprove = fscoreimprove + str(metricsDataPrediction[i][10])
        
        if(len(lonliness)):
            lonliness = lonliness + ','
        lonliness = lonliness + str(dataLonliness[i])
        

    waste = 'WASTE,' + 'num_partitions_' + str(config['num_partitions']) + ',' + str(scipy.mean(dataWaste)) + ',' + str(scipy.std(dataWaste)) + ',' + waste

    cutratio = 'CUT_RATIO,' + 'num_partitions_' + str(config['num_partitions']) + ',' + str(scipy.mean(dataCutRatio)) + ',' + str(scipy.std(dataCutRatio)) + ',' + cutratio
    ec = 'EC,' + 'num_partitions_' + str(config['num_partitions']) + ',' + str(scipy.mean(dataEC)) + ',' + str(scipy.std(dataEC)) + ',' + ec
    tcv = 'TCV,' + 'num_partitions_' + str(config['num_partitions']) + ',' + str(scipy.mean(dataTCV)) + ',' + str(scipy.std(dataTCV)) + ',' + tcv

    lonliness = "LONELINESS," + 'num_partitions_' + str(config['num_partitions']) + ',' + str(scipy.mean(dataLonliness)) + ',' + str(scipy.std(dataLonliness)) + ',' + lonliness
    
    qds = 'QDS,' + 'num_partitions_' + str(config['num_partitions']) + ',' + str(scipy.mean(dataQDS)) + ',' + str(scipy.std(dataQDS)) + ',' + qds
    conductance = 'CONDUCTANCE,' + 'num_partitions_' + str(config['num_partitions']) + ',' + str(scipy.mean(dataCOND)) + ',' + str(scipy.std(dataCOND)) + ',' + conductance
    maxperm = 'MAXPERM,' + 'num_partitions_' + str(config['num_partitions']) + ',' + str(scipy.mean(dataMAXPERM)) + ',' + str(scipy.std(dataMAXPERM)) + ',' + maxperm
    rbse = 'RBSE,' + 'num_partitions_' + str(config['num_partitions']) + ',' + str(scipy.mean(dataRBSE)) + ',' + str(scipy.std(dataRBSE)) + ',' + rbse

    nmi = 'NMI,' + 'num_partitions_' + str(config['num_partitions']) + ',' + str(scipy.mean(dataNMI)) + ',' + str(scipy.std(dataNMI)) + ',' + nmi

    fscore = "FSCORE," + 'num_partitions_' + str(config['num_partitions']) + ',' + str(scipy.mean(dataFscore)) + ',' + str(scipy.std(dataFscore)) + ',' + fscore
    fscoreimprove = "FSCORE_IMPROVE," + 'num_partitions_' + str(config['num_partitions']) + ',' + str(scipy.mean(dataFscoreImprove)) + ',' + str(scipy.std(dataFscoreImprove)) + ',' + fscoreimprove

    print(cutratio)
    print(ec)
    print(tcv)
    print(lonliness)
    print(qds)
    print(conductance)
    print(maxperm)
    print(rbse)
    print(nmi)
    print(fscore)
    print(fscoreimprove)

Mode 2
Mode 2 Iteration 0
Mode 2 Iteration 20
Mode 2 Iteration 40
Mode 2 Iteration 60
Mode 2 Iteration 80
Average Elapsed Time = 4.29238665819
CUT_RATIO,num_partitions_2,0.128856866361,0.0246361966576,0.136666666667,0.166197183099,0.115384615385,0.172890733057,0.139259259259,0.104294478528,0.146814404432,0.139605462822,0.113345521024,0.0842787682334,0.129682997118,0.0780346820809,0.177304964539,0.178674351585,0.13606557377,0.110707803993,0.113114754098,0.122448979592,0.123711340206,0.14657980456,0.0877483443709,0.1463878327,0.133603238866,0.112341772152,0.0950819672131,0.123641304348,0.128205128205,0.109546165884,0.149831649832,0.106177606178,0.189440993789,0.128427128427,0.14922813036,0.111459968603,0.127819548872,0.137221269297,0.100995732575,0.157094594595,0.144312393888,0.157798165138,0.1328125,0.139534883721,0.154471544715,0.129471890971,0.091684434968,0.112456747405,0.138138138138,0.133440514469,0.124,0.0979133226324,0.190851735016,0.114566284779,0.123050259965,0.109004739336,0.1

/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


Average Elapsed Time = 14.7964232588
CUT_RATIO,num_partitions_9,0.241441144733,0.0303667424214,0.28,0.250704225352,0.260869565217,0.295988934993,0.271111111111,0.220858895706,0.265927977839,0.247344461305,0.235831809872,0.176661264182,0.268011527378,0.239884393064,0.29609929078,0.278097982709,0.204918032787,0.239564428312,0.216393442623,0.267006802721,0.247422680412,0.224755700326,0.281456953642,0.269961977186,0.281376518219,0.238924050633,0.232786885246,0.254076086957,0.215384615385,0.219092331768,0.291245791246,0.222007722008,0.30900621118,0.215007215007,0.265866209262,0.183673469388,0.160401002506,0.214408233276,0.233285917496,0.282094594595,0.227504244482,0.266055045872,0.252604166667,0.280361757106,0.265040650407,0.248722316865,0.191897654584,0.237024221453,0.226726726727,0.229903536977,0.226,0.179775280899,0.260252365931,0.230769230769,0.253032928943,0.251184834123,0.232061068702,0.220994475138,0.230392156863,0.237017310253,0.253894080997,0.191553544495,0.231404958678,0.258620689

/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


Average Elapsed Time = 17.9380954337
CUT_RATIO,num_partitions_11,0.246060024213,0.0323001940822,0.275,0.223943661972,0.270903010033,0.300138312586,0.272592592593,0.255623721881,0.270083102493,0.251896813354,0.266910420475,0.173419773096,0.302593659942,0.225433526012,0.292553191489,0.293948126801,0.24262295082,0.230490018149,0.252459016393,0.284013605442,0.237113402062,0.257328990228,0.261589403974,0.287072243346,0.275303643725,0.224683544304,0.218032786885,0.263586956522,0.213675213675,0.226917057903,0.294612794613,0.235521235521,0.299689440994,0.194805194805,0.26243567753,0.182103610675,0.185463659148,0.246998284734,0.258890469417,0.256756756757,0.212224108659,0.260550458716,0.248697916667,0.257105943152,0.255284552846,0.27597955707,0.179104477612,0.273356401384,0.222222222222,0.231511254019,0.22,0.194221508828,0.288643533123,0.234042553191,0.294627383016,0.276461295419,0.251908396947,0.24861878453,0.240196078431,0.229027962716,0.247663551402,0.191553544495,0.24132231405,0.24294670846

...

Mode 12 Iteration 20


/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


Mode 12 Iteration 40
Mode 12 Iteration 60
Mode 12 Iteration 80


/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


Average Elapsed Time = 19.2717250466
CUT_RATIO,num_partitions_12,0.251410831468,0.0309293281529,0.266666666667,0.257746478873,0.304347826087,0.273858921162,0.277037037037,0.247443762781,0.272853185596,0.248861911988,0.254113345521,0.192868719611,0.302593659942,0.25289017341,0.285460992908,0.28818443804,0.240983606557,0.257713248639,0.259016393443,0.275510204082,0.230670103093,0.257328990228,0.251655629139,0.281368821293,0.291497975709,0.251582278481,0.222950819672,0.267663043478,0.222222222222,0.22848200313,0.274410774411,0.239382239382,0.302795031056,0.20202020202,0.257289879931,0.197802197802,0.177944862155,0.236706689537,0.28307254623,0.319256756757,0.230899830221,0.264220183486,0.255208333333,0.266149870801,0.266666666667,0.24531516184,0.215351812367,0.268166089965,0.217717717718,0.239549839228,0.216,0.178170144462,0.277602523659,0.237315875614,0.279029462738,0.27804107425,0.251908396947,0.246777163904,0.253267973856,0.202396804261,0.264797507788,0.248868778281,0.251239669421,0.236